In [5]:
## Dedependiecies 

# Web scrape
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import psycopg2

# SQL
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

# General
import pandas as pd
import numpy as np

## Web scraping

In [6]:
# Open browser with splinter 
# (Headless = True) Thus we do not see the browser oepning and closing
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [7]:
# HTML Object and Beautiful Soup Parseing
url = 'https://www.nei.org/resources/statistics/state-electricity-generation-fuel-shares'
browser.visit(url)

In [8]:
# HTML Object and Beautiful Soup Parseing
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [21]:
# Retrieve and collect all desired data
webTable = soup.find_all('table')

# All desired data is in one line, thus have to split
us_percent_list = []
temp_list = []

for result in webTable:
    tableData = []
    us_percent_list.append([])
    tableData = result.find_all('td')
    
#print(us_percent_list) 
print(f'Length of scraped data: {len(tableData)}')
print(tableData[509].text) 

Length of scraped data: 510
0.2


In [22]:
# Retrieve and collect all desired data
webTableRows = soup.find_all('tr')

# All desired data is in one line, thus have to split
us_percent_list = []
temp_list_small = []
temp_list_big = []

for tr in webTableRows:
    tableData = tr.find_all('td')
    tableColumnHeader = tr.find_all('th')
    us_percent_list= []

    for td in tableData:
        data = td.text
        temp_list_small.append(data)
        temp_list_big.append(temp_list_small)
        temp_list_small = []

    us_percent_list.append(temp_list_big)
print(us_percent_list)

[[['Alabama'], ['31.4'], ['19.2'], ['38.7'], ['0.0'], ['7.8'], ['0.0'], ['0.3'], ['0.0'], ['2.5'], ['Alaska'], ['0.0'], ['10.6'], ['49.9'], ['14.4'], ['22.0'], ['0.0'], ['0.0'], ['2.4'], ['0.6'], ['Arizona'], ['28.1'], ['20.4'], ['40.9'], ['0.1'], ['5.4'], ['0.0'], ['4.5'], ['0.5'], ['0.2'], ['Arkansas'], ['22.0'], ['37.9'], ['33.0'], ['0.1'], ['4.5'], ['0.0'], ['0.3'], ['0.0'], ['2.2'], ['California'], ['7.9'], ['0.1'], ['42.4'], ['0.0'], ['19.5'], ['5.6'], ['14.0'], ['7.3'], ['3.2'], ['Colorado'], ['0.0'], ['45.1'], ['30.4'], ['0.0'], ['2.5'], ['0.0'], ['2.1'], ['19.4'], ['0.4'], ['Connecticut'], ['41.7'], ['0.1'], ['52.8'], ['0.2'], ['1.3'], ['0.0'], ['0.4'], ['0.0'], ['3.3'], ['Delaware'], ['0.0'], ['2.3'], ['94.7'], ['0.8'], ['0.0'], ['0.0'], ['1.0'], ['0.1'], ['1.0'], ['District of Columbia'], ['0.0'], ['0.0'], ['27.1'], ['0.0'], ['0.0'], ['0.0'], ['15.1'], ['0.0'], ['57.8'], ['Florida'], ['11.8'], ['9.1'], ['74.3'], ['0.1'], ['0.1'], ['0.0'], ['1.6'], ['0.0'], ['3.1'], ['Georgia

In [ ]:
rows = soup.find("table").find("tbody").find_all("tr")
#, border=1
for row in rows:
    cells = row.find_all("td")
    rn = cells.text()
print (rn)

In [ ]:
# Convert data into df and clean
#US_percentage_df = pd.DataFrame(us_percent_list, columns = tableHeaders)
US_percentage_df = pd.DataFrame(us_percent_list, columns = ["State", "Nuclear (%)", "Coal (%)", "Natural Gas (%)", 
                                                            "Petroleum (%)", "Hydro (%)", "Geothermal (%)", 
                                                            "Solar - PV (%)", "Wind (%)", "Biomass and Other (%)"])


print(f'The length of this df is: {len(US_percentage_df)}')
US_percentage_df.head()

#### ValueError: 10 columns passed, passed data had 510 columns  ####

In [ ]:
##### Possible needed code for web scraping #######
# Convert years to integers 
# insider_df['Year'] = (insider_df['Year']).astype(int)

# Removing extra years 
#insider_df = insider_df[insider_df['Year'] <= 2019]
#insider_df = insider_df[insider_df['Year'] >= 2010]
#insider_df = insider_df.sort_values(by=['Year'])





#### Dont forget to quit it

In [24]:
# Finished scraping from the web, quit browser in background
browser.quit()

### CSV Load 

In [ ]:
## Source 1
# read from local file
raw_csv_1 = "./Resources/name1.csv"
CSV_df_1 = pd.read_csv(raw_csv_1, sep = ',', encoding='ISO-8859-1')

# Display head of data set to be sure it was loaded correctly
CSV_df_1.head()

In [ ]:
## Source 2
# read from local file
raw_csv_2 = "./Resources/name2.csv"
CSV_df_2 = pd.read_csv(raw_csv_1, sep = ',', encoding='ISO-8859-1')

# Display head of data set to be sure it was loaded correctly
CSV_df_2.head()

In [ ]:
## Source 3
# read from local file
raw_csv_3 = "./Resources/name3.csv"
CSV_df_3 = pd.read_csv(raw_csv_1, sep = ',', encoding='ISO-8859-1')

# Display head of data set to be sure it was loaded correctly
CSV_df_3.head()

## Add CSV and json to SQL, Postgress

In [47]:
# Connect to local database
code_source_passcode = open('/Users/Richa/Desktop/Files/SQL_private_connect.py') 
# Add you password here
#code_source_passcode = open('/Users/......./SQL_private_connect.py') 

sql_private_connect = code_source_passcode.read()

In [48]:
engine = create_engine(f'postgresql://{sql_private_connect}@localhost:5432/Need_To_Add_(name)_db')
connection = engine.connect()

In [49]:
# Check for tables
engine.table_names() 

['combined_music', 'insider', 'billboard_top', 'spotify']

In [50]:
# Use pandas to load json converted DataFrame into database
Web_Scraped_df.to_sql(name='us_percentage', 
                      con=engine, 
                      if_exists='replace', 
                      index=False)


NameError: name 'Web_Scraped_df' is not defined

In [ ]:
# Use pandas to load csv converted DataFrame into database
CSV_df_1.to_sql(name='name1', 
               con=engine, 
               if_exists='replace', 
               index=False)

In [ ]:
# Use pandas to load csv converted DataFrame into database
CSV_df_2.to_sql(name='name2', 
               con=engine, 
               if_exists='replace', 
               index=False)

In [ ]:
# Use pandas to load csv converted DataFrame into database
CSV_df_3.to_sql(name='name3', 
               con=engine, 
               if_exists='replace', 
               index=False)

In [ ]:
# Check for tables
engine.table_names() 

### Check tables and data before making charts

In [ ]:
# Confirm data has been added by querying the insider table
SQL_CSV1_loaded = pd.read_sql_query('select * from name1', con=engine)
SQL_CSV1_loaded.head()

In [ ]:
# Confirm data has been added by querying the insider table
SQL_CSV1_loaded = pd.read_sql_query('select * from name1', con=engine)
SQL_CSV1_loaded.head()

In [ ]:
# Confirm data has been added by querying the insider table
SQL_CSV1_loaded = pd.read_sql_query('select * from name1', con=engine)
SQL_CSV1_loaded.head()

In [ ]:
# Confirm data has been added by querying the insider table
SQL_CSV1_loaded = pd.read_sql_query('select * from name1', con=engine)
SQL_CSV1_loaded.head()